# Análise Sentimental do Review de Jogos

## Projeto da disciplina **SSC0287 - Mineração de Dados Não Estruturados**

- Lucas Ivars Cadima Ciziks - luciziks@usp.br - 12559472

- Gustavo Silva de Oliveira - guspfc03@usp.br - 12567231

---

# 1. Introdução

O objetivo principal desse trabalho é **sumarizar informações textuais** no contexto de **Review de Jogos por Usuários**, utilizando as técnicas de Mineração de Dados Textuais aprendidas no decorrer da disciplina. Dentro de plataformas de games, como a Steam e Epic Games, é comum cada jogo ter uma seção contendo a avaliação geral dos usuários que já jogaram. Desse modo, novos compradores podem ter uma ideia do que esperar de um jogo. 


Nesse contexto, nossa abordagem visa extrair o sentimento geral na opinião de um usuário ao descrever sua experiência jogando determinado jogo, verificando, assim, se a avaliação foi **Positiva**, **Negativa** ou **Neutra**. Para isso, treinamos um modelo base do [BERT](https://huggingface.co/bert-base-uncased) com uma base de avaliações de jogos advindas do Twitter. Desse modo, o modelo é capaz de analisar a linguagem complexa presente nos reviews e traduzi-lá em informação resumida.

Além disso, treinamos um segundo modelo dedicado à detecção de discurso de ódio, classificando o texto como **Discurso de Ódio**, **Linguagem ofensiva** ou **Nada Detectado** nos reviews. Assim, em um caso real como na plataforma Steam, o uso combinado desses modelos permite coletar informações diretas sobre os jogos, além de evitar a disseminação de conteúdos prejudiciais nesses fóruns.

Com perspectiva para futuras melhorias, propomos a implementação da explicabilidade dos modelos. Esse aprimoramento permitirá uma compreensão mais clara sobre os motivos específicos que contribuem para o sentimento geral ser positivo, negativo ou neutro. Ademais, identificar quais palavras específicas foram a razão para se qualificar a review como discurso de ódio ou linguagem ofensiva seria útil para eventuais censuras ou aviso ao usuário.

O projeto foi desenvolvido na linguagem **Python**, utilizando o [Poetry](https://python-poetry.org) para gerenciamento das dependências. Dentre as bibliotecas utilizadas, estão:

* [Pandas](https://pandas.pydata.org);
* [Scikit-learn](https://scikit-learn.org/stable/);
* [Keras](https://keras.io);
* [HuggingFace](https://huggingface.co/docs/huggingface_hub/index).

Antes de rodar o notebook, certifique-se de instalar todas as depências através do comando:

```poetry install```

---

# 2. Pré-Processamento

In [ ]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant

---

# 3. Extração de Padrões

---

# 4. Pós-Processamento

---

# 5. Uso do Conhecimento

A fim utilizar os modelos de **Análise de Sentimento** e **Detecção de Discurso de Ódio** treinados nas etapas anteriores, construímos um site para coletar a opinião do usuário acerca dos nomeados a Game of The Year 2023 e verificar o sentimento geral e se houve discurso de ódio na descrição.

Para isso, utilizamos o [Streamlit](https://docs.streamlit.io), uma biblioteca do Python que facilita a implementação de uma interface gráfica para projetos de Machine Learning e Data Science. O código está disponível no arquivo `app.py` e é possível executá-lo localmente através do comando:

```poetry run streamlit run app.py```
